<a href="https://colab.research.google.com/github/ChitranshS/HiringChallenge-1/blob/main/Hiring_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we have to check whether we are connected to the right runtime or not.
## Please connect to the T4 runtime before moving ahead.
I will also add the instructions to the github as well

In [ ]:
#@title GPU available?
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-9217c8fc-684c-494a-34e4-afc475040a44)


### We need to download approx 5-6 GBs in this step
The next step will take some while as we need to download a whole bunch of libraries in a single go.


In [ ]:
#@title Initial Library Setup
import os, subprocess

def setup():
    install_cmds = [
        ['pip', 'install', 'gradio'],
        ['pip', 'install', 'open_clip_torch'],
        ['pip', 'install', 'clip-interrogator'],
        ['pip', 'install', 'together'],
        ['pip','install','gradio_client'],
        ['pip','install','pytesseract']
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

import gradio as gr
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = 'ViT-H-14/laion2b_s32b_b79k'  # Default clip model
config.caption_model_name = 'blip-large'  # Default caption model
ci = Interrogator(config)
os.mkdir("/content/uploads")
os.mkdir("/content/augment")



In [ ]:
#@title CLIP Implementation
def image_to_prompt(image):
    ci.config.chunk_size =1024
    ci.config.flavor_intermediate_count =1024
    # ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')
    modes = ['best','fast','normal']
    mode = random.choice(modes)  # Set mode to 'fast' by default
    generated_prompt = ci.interrogate_fast(image)
    return generated_prompt


In [ ]:
#@title Upload Image (PNG/JPEG only for now)
import gradio as gr

def prompt_tab():
    global stored_prompt  # Declare the stored_prompt variable as global

    with gr.Column():
        with gr.Row():
            print(1)
            image = gr.Image(type='pil', label="Image")
            print(2)
        prompt = gr.Textbox(label="Prompt")
        print(3)
        button = gr.Button("Generate prompt")
        print(4)
        button.click(image_to_prompt, inputs=[image], outputs=[prompt])
        print(5)

            # Store the prompt value after rendering
        prompt.change(update_stored_prompt, inputs=[prompt], outputs=[])


def update_stored_prompt(prompt):
    global stored_prompt
    stored_prompt = prompt  # Assign the prompt value to the stored_prompt variable

with gr.Blocks() as ui:
    with gr.Tab("Prompt"):
        stored_prompt = ""  # Initialize the stored_prompt variable
        prompt_tab()
    ui.launch()


1
2
3
4
5
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f7c2594d60e9d82bd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
stored_prompt

'a close up of a diamond ring with a diamond band, hexagonal ring, “diamonds, centered thin ring iris, h 576, magic ring with a diamond, diamond and rose quartz, alp, h 768, h 7 6 8, engineered, # e 5 3 7 1 b, ƒ/5.6, elden ring style'

In [ ]:
user_prompt = " Shutter Camera,Film,DSLR"

In [ ]:
#@title Generate Dynamic Prompt
import os
import random
import together

together.api_key = ""
response = together.Complete.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    prompt=f"""ONLY REPLY AFTER READING THE ENTIRE TEXT NOT BEFORE THAT.THIS IS NOT A CONVERSATION.REPLY AS IF YOU HAVE TO A API CALL EXPECTING A NON CONVERSATIVE ANSWER.I will give you a product description i.e. {stored_prompt}
    Identify entities which will look appealing and aesthetic in a marketing campaign studio where the product  is :{user_prompt}
    Now your task is to generate keywords from this and {user_prompt} which are related to the product and can be used in its marketing.
    ONLY GIVE THE PROMPT TEXT NOTHING ELSE. THE PROMPT SHOULD BE COMPLETE AND LESS THAN 50 WORDS. DON'T REPEAT THE PROMPT IN A SINGLE REPLY. STOP AND LEAVE BLANK SPACES WHEN THE PROMPT IS OVER. DO NOT USE '\n' AND OTHER MARKDOWN """,
    temperature = round(random.uniform(1, 2),1),
    top_k = 50,
    max_tokens = 300,
    top_p = 0.7,
    repetition_penalty = 1
)
print(response)


<ipython-input-678-557a96e74890>:7: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/usr/local/lib/python3.10/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


{'id': '87a2ae861e3a6039-SIN', 'object': <ObjectType.Completion: 'text.completion'>, 'created': 1714093712, 'model': 'meta-llama/Llama-3-70b-chat-hf', 'choices': [{'index': 0, 'logprobs': None, 'finish_reason': <FinishReason.Length: 'length'>, 'text': " AND HTML TAGS AND DO NOT LEAVE YOUR OWN OPINION.  JUST PROVIDE THE KEYWORDS.\n\n\ncamera \nlens \nblack \nbackground \n80s \n35mm \nfilm \nmedium \nformat \ntri \nx \npan \nstock \nphoto \nrealistic \nnikon \nmamiya \nShutter \nCamera \nFilm \nDSLR \nbeach \n\ncamera lens \nfilm camera \nmedium format camera \n35mm film camera \ndslr camera \nbeach camera \nfilm photography \ncamera photography \nphotography studio \nbeach photography \nfilm camera style \nretro camera \nvintage camera \nanalog camera \nclassic camera \nold camera \nphotography equipment \ncamera accessories \nfilm stock \nphotography props \nbeach props \nphotography studio props \nfilm camera accessories \nvintage photography \nretro photography \nanalog photography \

In [ ]:
generation_prompt = response['choices'][0]
generation_prompt

{'index': 0,
 'logprobs': None,
 'finish_reason': <FinishReason.Length: 'length'>,
 'text': " AND HTML TAGS AND DO NOT LEAVE YOUR OWN OPINION.  JUST PROVIDE THE KEYWORDS.\n\n\ncamera \nlens \nblack \nbackground \n80s \n35mm \nfilm \nmedium \nformat \ntri \nx \npan \nstock \nphoto \nrealistic \nnikon \nmamiya \nShutter \nCamera \nFilm \nDSLR \nbeach \n\ncamera lens \nfilm camera \nmedium format camera \n35mm film camera \ndslr camera \nbeach camera \nfilm photography \ncamera photography \nphotography studio \nbeach photography \nfilm camera style \nretro camera \nvintage camera \nanalog camera \nclassic camera \nold camera \nphotography equipment \ncamera accessories \nfilm stock \nphotography props \nbeach props \nphotography studio props \nfilm camera accessories \nvintage photography \nretro photography \nanalog photography \nclassic photography \nold photography \nphotography tools \ncamera tools \nfilm tools \nbeach tools \nphotography studio tools \nfilm camera tools \nvintage ph

In [ ]:
# !pip install gradio_client

In [ ]:
# # !pip install pytesseract
# round(random.uniform(1, 2),1)

In [ ]:
# # Install Tesseract
# !sudo apt install tesseract-ocr

# # Install pytesseract, a Python wrapper for Tesseract
# !pip install pytesseract


In [ ]:
# import pytesseract
# from PIL import Image
# import numpy as np
# import cv2
# import matplotlib.pyplot as plt
# imageSent = Image.open('/content/uploads/product3.png')
# image_cv = np.array(imageSent)

# # Convert RGB to BGR (OpenCV uses BGR by default)

# # Apply Canny edge detection
# edges = cv2.Canny(image_cv, 100, 1100)  # These thresholds can be adjusted

# # Convert back to PIL image to use any PIL functionalities further or just display using matplotlib
# edges_image = Image.fromarray(edges)
# extracted_text = pytesseract.image_to_string(edges)
# plt.imshow(edges_image)
# print(extracted_text)

# Change this number as per the product{number}


In [ ]:
number = 1

In [ ]:
#@title Generate Product Background
from gradio_client import Client

client = Client("https://hysts-controlnet-v1-1.hf.space/")
imageSent = Image.open(f'/content/uploads/product{number}.png')
rotated_image = imageSent.rotate(random.uniform(-10,10))
# rotated_image = imageSent.transpose(Image.FLIP_LEFT_RIGHT)

save_path = f'/content/augment/product{number}.png'  # Replace with your output file path and desired format
rotated_image.save(save_path, 'PNG')

result = client.predict(
		f"/content/augment/product{number}.png",	# str (filepath on your computer (or URL) of image) in 'parameter_8' Image component
		generation_prompt,	# str  in 'Prompt' Textbox component
		f"best quality, extremely detailed , {user_prompt}, Marketing , Product photography, enhanced , Multiple angles",	# str  in 'Additional prompt' Textbox component
		"longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality,dull colours",	# str  in 'Negative prompt' Textbox component
		1,	# int | float (numeric value between 1 and 1) in 'Number of images' Slider component
		720.00,	# int | float (numeric value between 256 and 768) in 'Image resolution' Slider component
		random.randint(30,96),	# int | float (numeric value between 1 and 100) in 'Number of steps' Slider component
		round(random.uniform(10,28),1),	# int | float (numeric value between 0.1 and 30.0) in 'Guidance scale' Slider component
		random.randint(0,2232232),	# int | float (numeric value between 0 and 2147483647) in 'Seed' Slider component
		50,	# int | float (numeric value between 1 and 255) in 'Canny low threshold' Slider component
		200,	# int | float (numeric value between 1 and 255) in 'Canny high threshold' Slider component
		api_name="/canny"
)
print(result)

Loaded as API: https://hysts-controlnet-v1-1.hf.space/ ✔
/tmp/gradio/e603a20c-c18f-42de-8900-f53d8c1ee399


In [ ]:
import os
for dirpath, dirnames, filenames in os.walk(result):
  if dirnames:
   directories = dirnames
   break
result

'/tmp/gradio/e603a20c-c18f-42de-8900-f53d8c1ee399'

In [ ]:
#@title Image Resizing
from PIL import Image

# Load the image
imageResize = Image.open(f'../content/augment/product{number}.png')
background1 = Image.open(f'../{result}/{directories[0]}/image.png')


# Specify the new dimensions
new_width , new_height = background1.size

# Resize the image
resized_image = imageResize.resize((new_width, new_height), Image.LANCZOS)

# Save the resized image
resized_image.save('resized_image.png')

# Display the resized image (optional)
resized_image.show()


In [ ]:
#@title Using the generated image and the mask

from PIL import Image

# Open the background and overlay images
background1 = Image.open(f'../{result}/{directories[0]}/image.png')
background2 = Image.open(f'../{result}/{directories[1]}/image.png')
overlay = Image.open('../content/resized_image.png')

# You might need to resize the overlay or use it as is, depending on the need
# For example, resize overlay to match background's size
# overlay = overlay.resize(background.size, Image.ANTIALIAS)
print("Background mode:", background1.mode)
print("Background mode:", background2.mode)
print("Overlay mode:", overlay.mode)
# Position the overlay at the desired position (top-left corner in this example)
# You can change (0, 0) to whatever coordinates you want
position = (0, 0)  # Position can be changed based on where you want to place the overlay
if background1.mode != 'RGBA':
    background = background1.convert('RGBA')
if background2.mode != 'RGBA':
    background = background2.convert('RGBA')
if overlay.mode != 'RGBA':
    overlay = overlay.convert('RGBA')
# Assuming both images are now RGBA
# Extract the alpha channel from the overlay
alpha_mask = overlay.split()[3]

# Paste the overlay using the alpha channel as a mask
background1.paste(overlay, (0, 0), alpha_mask)  # Adjust position as needed

# Save or show the result
background1.save('output_image1.png')  # Save as PNG to preserve transparency in the output
background1.show()

background2.paste(overlay, (0, 0), alpha_mask)  # Adjust position as needed

# Save or show the result
background2.save('output_image2.png')  # Save as PNG to preserve transparency in the output
background2.show()

Background mode: RGB
Background mode: RGB
Overlay mode: RGBA


## Now two images will be there output_image1.png and output_image2.png
One of them is the canny mask and other is the final product.